### Adult Dataset

The Adult Census Income dataset consists of demographic information about individuals. This data was extracted from the 1994 Census bureau database by Ronny Kohavi and Barry Becker (Data Mining and Visualization, Silicon Graphics) and in addition, this specific dataset is downloaded from kaggle. This dataset includes multiple variables that could potentially influence income, which is the target variable. Here’s a summary of the variables:

- Age: The age of the individual

- Workclass: Type of WorkSector

- fnlwgt: weighted tallies of any specified socio-economic characteristics of the population.

- education: Education Level (Categorical)

- Education.num: Education-level (numeric)

- Martial.status: Martial Status of that individual

- Occupation: Current job of that individual

- Relationship: Relationship Status of that individual

- Race: Race of that individual

- Sex: Gender of that individual

- Capital.gain: Capital gains (Additional income from income)

- Capital.loss: Capital losses

- Hours.per.week: Hours per week an individual works for

- Native.country: Country of origin for that individual

- Income: Income category indicating if the individual earns more or less than $50K

It seems like there are 32651 rows and 15 columns which is a pretty good size for an analysis. Some variables also contain "?" which indicate missing values on some rows. 

### Question Of Interest

Is there a significant income disparity between male and female individuals, and how do other factors like occupation and hours worked per week influence this disparity?

Response Variable: Income

Explanatory Variables:

- sex: This variable directly addresses gender differences in income.
- occupation: Some occupations might have different earning potential based on gender.
- hours.per.week: Examining if hours worked per week contribute to income disparities between genders.


This question focuses on inference to understand the factors influencing gender-based income disparities, providing insights into whether these disparities persist across occupations and work hours.

### Importing Libraries

In [1]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(ggplot2)
library(leaps)
library(glmnet)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-8



In [2]:
adult_df <- read_csv("data/adult.csv")

head(adult_df)
dim(adult_df)

ERROR: Error: 'data/adult.csv' does not exist in current working directory ('/home/jovyan/work/Stat301_Project').


In [ ]:
summary(adult_df)

In [ ]:
colSums(is.na(adult_df))

In [ ]:
colSums(adult_df=="?")

It seems like there are missing values in workclass, occupation and the native country values. We can input the correct ratio of the categorical variables to ensure we maintain the spread of the data, thus using proportional imputation.

In [ ]:
fill_with_proportion <- function(column) 
{
  non_missing <- column[column != "?"]
  value_counts <- table(non_missing)
  proportions <- prop.table(value_counts)
  
  column[column == "?"] <- sample(names(proportions), sum(column == "?"), replace = TRUE, prob = proportions)
  
  return(column)
}

adult_df$workclass <- fill_with_proportion(adult_df$workclass)
adult_df$occupation <- fill_with_proportion(adult_df$occupation)
adult_df$native.country <- fill_with_proportion(adult_df$native.country)

colSums(adult_df=="?")

Now, we should check for duplicate rows.

In [ ]:
duplicates <- adult_df[duplicated(adult_df), ]

if (nrow(duplicates) > 0) {
  print(duplicates)
} else {
  print("No duplicate rows found.")
}

Seems like there are 24 rows of duplicated data. This may be an input error from employees doing data entry or the system having errors. We will remove all of them.

In [ ]:
adult_df <- distinct(adult_df)
dim(adult_df)

Now, we are ready to do EDA on this dataset.

### EDA

In [ ]:
income_sex_barplot <- ggplot(adult_df, aes(x = sex, fill = income)) +
                      geom_bar(position = "dodge") +
                      labs(title = "Income Distribution by Gender", x = "Gender", y = "Count") 

income_sex_barplot

It appears that a larger proportion of males earn more than $50K compared to females. In addition, females have a higher count in the <=50K income category than in the >50K category, suggesting a difference in ratio.

In [ ]:
adult_df_proportions <- adult_df |>
  group_by(occupation, sex, income) |>
  summarise(count = n(), .groups = 'drop') |>
  mutate(proportion = count / sum(count))

occupation_sex_income_plot <- ggplot(adult_df_proportions, aes(x = occupation, y = proportion, fill = income)) +
                              geom_bar(stat = "identity", position = "dodge") +
                              facet_wrap(~sex) +
                              labs(title = "Income Distribution by Occupation and Gender", x = "Occupation", y = "Count") +
                              theme(axis.text.x = element_text(angle = 45, hjust = 1))
occupation_sex_income_plot

This bar graph shows the distribution of income through different occupations between male and female individuals. Males tend to have a higher proportion of individuals earning more than 50K compared to females across most occupations such as "Exec-managerial". This suggests potential gender disparities in income. In addition, there are some occupations like "Handlers-cleaners," "Other-service," and "Priv-house-serv" have a predominantly larger proportion of individuals earning less than 50K for both genders. We need to examine more relationships between men and women income levels.

In [ ]:
hours_per_week_sex_plot <- ggplot(adult_df, aes(x = income, y = hours.per.week)) +
                           geom_boxplot() +
                           labs(title = "Hours Per Week based on Income and Gender", x ="Income Level", y = "Hours Per Week") +
                           facet_wrap(~sex) 
hours_per_week_sex_plot             

In [ ]:
adult_df <- adult_df %>%
  mutate(education = fct_relevel(education, 
                                 "Preschool", "1st-4th", "5th-6th", "7th-8th", 
                                 "9th", "10th", "11th", "12th", "HS-grad", 
                                 "Some-college", "Assoc-acdm", "Assoc-voc", 
                                 "Bachelors", "Masters", "Doctorate"))

education_sex_income_plot <- ggplot(adult_df, aes(x = education, fill = income)) +
                              geom_bar(position = "dodge") +
                              facet_wrap(~sex) +
                              labs(title = "Income Distribution by Education and Gender", x = "Education", y = "Count") +
                              theme(axis.text.x = element_text(angle = 45, hjust = 1))
education_sex_income_plot

It seems for education, it looks around the same for both female and male bar graphs where we can see a trend where the higher the education level, the higher the proportions there is for 50k or greater income levels. 

### Methods and Plan

Before proprosing a model, we would first remove repetitive features that represent very identical factors to predict our response variable. For example, we have education and education level which, if we add both variables to our model, will only introduce more noise than predictive power. In addition, we know these two features have multicollinearity, thus removing one of them would be good. 

For our analysis of the Adult Census Income dataset, I propose using a Logistic Regression model to predict whether an individual's income exceeds $50K annually. Logistic regression is a robust and interpretable method for binary classification tasks, aligning well with our target variable, which has two categories: <=50K and >50K. 

#### Why This Method Is Appropriate

The target variable is binary, and logistic regression is designed to model the probability of an outcome as a function of explanatory variables. It allows us to interpret the relationship between predictor variables (e.g., age, hours.per.week, occupation) and the odds of earning more than $50K. Logistic regression is computationally efficient and scalable for a dataset of this size, which contains over 30,000 observations.


#### Variable Selection Method: Backward Selection

To select the most relevant predictors for the model, I propose using Backward Selection. This approach starts with all potential explanatory variables included in the model. Variables are removed one at a time based on a criterion such as p-values, until the best-fitting model is achieved. Below are reasons why backward selection is appropriate:

1. Exploratory Dataset: Backward selection is useful when starting with all variables makes sense, as we want to explore which predictors do not contribute significantly to the model.
2. Handling Redundancy: Backward selection helps identify and remove irrelevant or redundant predictors that may dilute the model’s accuracy or interpretability.
3. Simplicity: By iteratively reducing the number of variables, backward selection ensures the final model is parsimonious without overlooking complex interactions.

#### Assumptions of Logistic Regression

1. Binary Response: The response variable is dichotomous (two possible responses) or the sum of dichotomous responses.
2. Independence: The observations must be independent of one another.
3. Variance Structure: As response is biomial, the variance is np * (1 - p) by definition and the highest variance it can reach is when p = 0.5.
4. Linearity: The log odds $(\frac{p}{1 - p})$ must be a linear function of the predictor


#### Potential Limitations of the method itself

1. Computational Cost: Backward selection can be computationally expensive for datasets with many predictors, as it starts with all variables.
2. Risk of Overfitting: Removing variables based on p-values or AIC alone might lead to overfitting, especially in small samples.
3. Missed Interactions: Backward selection does not automatically explore variable interactions, which may reduce model performance.



### Implementation of the Proposed Model

In [ ]:
head(adult_df)

Removing these Features: 

- fnlwgt: I personaly do not know how they calculate this feature and in additon, it is not a characteristic of individuals but rather reflects sampling probabilities or demographic weights thus does not have a direct relationship with an individual's income level.
- education.num: We already have education, thus, this feature would just introduce noise to the model
- race: It might introduce ethical concerns or bias in the model which we do not want our model to be trained on.

In [ ]:
adult_df <- adult_df |>
select(-fnlwgt, -education.num, -race)

In [ ]:
adult_df$income <- ifelse(adult_df$income == ">50K", 1, 0)

head(adult_df)

#### Splitting the dataset into train and testing portions

In [ ]:
set.seed(123)

adult_df$income <- as.factor(adult_df$income)

unique_levels <- sapply(adult_df, function(x) length(unique(x)))
print(unique_levels)


adult_df <- adult_df[, unique_levels > 1]

training_adult <- adult_df %>%
  sample_frac(0.6)

testing_adult <- adult_df %>%
  setdiff(training_adult)

#### Backward Selection

In [ ]:
adult_backward_sel <- regsubsets(
  x = income ~ .,                
  data = training_adult,           
  nvmax = ncol(training_adult) - 1,                    
  method = "backward"             
)

adult_backward_summary <- summary(adult_backward_sel)
best_model_size <- which.max(adult_backward_summary$adjr2)

best_model_coefs <- coef(adult_backward_sel, id = best_model_size)

best_model_coefs_df <- as.data.frame(best_model_coefs)
colnames(best_model_coefs_df) <- "Coefficient"

cat("Coefficients for the best model:\n")
print(best_model_coefs_df)

#### Logistic Regression

In [ ]:
final_predictors <- make.names(names(best_model_coefs)[-1])
final_predictors

Since we see parts of education, occupation, relationship and capital gain and capital loss to be the most important variables, we will use them for our final model. An interesting observation as well is that sex is not a part of the most important variable.

In [ ]:
final_model <- glm(income ~ education + marital.status +  occupation + relationship + capital.gain + capital.loss, data = training_adult, family = binomial)

summary(final_model)

In [ ]:
predicted_probs <- predict(final_model, newdata = testing_adult, type = "response")

# Convert probabilities to class predictions (threshold = 0.5)
predicted_classes <- ifelse(predicted_probs > 0.5, 1, 0)

# Create a confusion matrix
confusion_matrix <- table(Actual = testing_adult$income, Predicted = predicted_classes)
cat("Confusion Matrix:\n")
print(confusion_matrix)

# Calculate accuracy
accuracy <- sum(diag(confusion_matrix)) / sum(confusion_matrix)
cat("Accuracy:", round(accuracy, 4), "\n")

## Discussion(initial)

This project looked at whether there are income differences between men and women and how factors like job type and hours worked per week play a role. Using a logistic regression model, we found that factors such as education level, marital status, job type, relationship status, capital gains, and capital losses were the most important predictors of income (>50K). Surprisingly, gender itself wasn’t one of the key factors in the final model.

Our initial analysis showed that men are more likely to earn over $50K than women. However, when other factors like education and job type were considered, gender’s influence on income became less important. This suggests that income differences might be more about structural issues, like the kinds of jobs or education levels people have, rather than gender alone.

The model had an accuracy of 83.32%, meaning it performed well in predicting income. However, some issues, like imbalanced data (more people earning less than $50K) and extreme values in capital gains and losses, might have affected the results. Improving the model by adding interactions between variables or trying other methods like random forests could make it better.

Future studies could explore why certain jobs or education levels have such a big impact on income. Adding more data about workplace policies or cultural factors could also help us understand income differences better. These findings are a step toward identifying ways to reduce income inequalities.